## Import library

In [1]:
!pip install EMD-signal
from scipy import stats
from scipy import fftpack
from PyEMD import EMD, EEMD # https://pyemd.readthedocs.io/en/latest/usage.html
from scipy.signal import find_peaks
from scipy.stats import kurtosis, skew
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
from sklearn.preprocessing import minmax_scale
import pandas as pd
import numpy as np
import statistics
import pickle
import math
import cmath
import os
import math 
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import tensorflow as tf
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers import Dense, Activation,Conv2D,MaxPooling2D, Dropout, Flatten
from sklearn.model_selection import *
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
import pandas as pd
import os
from sklearn.feature_selection import SelectKBest, SelectPercentile, mutual_info_classif
from sklearn.neural_network import MLPClassifier
from numpy import mean


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 6.9 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.66.1
    Uninstalling tqdm-4.66.1:
      Successfully uninstalled tqdm-4.66.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
beatrix-jupyterlab 2023.621.222118 requires jupyter-server~=1.16, but you have jupyter-server 2.6.0 which is incompatible.
fitter 1.6.0 requires tqdm<5.0.0,>=4.65.1, but you have tqdm 4.64.1 which is incompatible.
momepy 0.6.0 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
ydata-profiling 4.3.1 requires scipy<1.11,>=1.4.1, but you have scipy 1.11.2 which is incompatible.


## IMF Creation

In [2]:
def cal_IMF(start, channel, data): 
    # print(data.shape)
    sample_rate = 256
    seconds = 10
    num_samples = sample_rate*seconds
    time_vect = (data.iloc[start:start+num_samples, channel])
    time_vect = np.array(time_vect)
    emd = EMD()
    imfs = emd.emd(time_vect)
    return imfs

## Fluction Index Calculation

In [ ]:
def fluctuation_index(x):
    sum = 0
    # print(x.shape[0]-1)
    for i in range(0, x.shape[0]-1):
        sum+=abs(x[i+1]-x[i])
        # print(sum)
    return sum/(x.shape[0]-1)

## SODP Calculation

In [5]:
def calc_area_of_sodp(X,Y,i,channel):
        #Area of Second Order Difference Plot
        SX = math.sqrt(np.sum(np.multiply(X,X))/len(X))
        SY = math.sqrt(np.sum(np.multiply(Y,Y))/len(Y))
        SXY = np.sum(np.multiply(X,Y))/len(X)
        D = cmath.sqrt((SX*SX) + (SY*SY) - (4*(SX*SX*SY*SY - SXY*SXY)))
        # print(D)
        a = 1.7321 *cmath.sqrt(SX*SX + SY*SY + D)
        b = 1.7321 * cmath.sqrt(SX*SX + SY*SY - D)
        Area = math.pi *a *b
        # print(SX,SY,SXY, D, a, b, Area)
        # print("Channel=  ",channel,"Area of SODP of IMF number= ",i, " is ", Area.real, " ", Area.imag)
        return Area.real

In [6]:
def SODP(y, i, channel):
        #remove outliers
        upper_quartile = np.percentile(y,80)
        lower_quartile = np.percentile(y,20)
        IQR = (upper_quartile - lower_quartile) * 1.5
        quartileSet = (lower_quartile- IQR, upper_quartile +IQR)
        y = y[np.where((y >= quartileSet[0]) & (y <= quartileSet[1]))]
        
        #plotting SODP
        # X = []
        # Y= []
        # for n in range(0, y.shape[0]-2):
        #     X.append(y[n+1]-y[n])
        #     Y.append(y[n+2]-y[n+1])
        X = np.subtract(y[1:],y[0:-1]) #x(n+1)-x(n)
        Y = np.subtract(y[3:],y[0:-3]).tolist()#x(n+2)-x(n-1)
        Y.extend([0])
        Y.extend([0])
        # plt.figure(figsize=(12, 6))
        # plt.scatter(X, Y, s=80, facecolors='none', edgecolors='b')
        # plt.show()
        # self.save_fig(X,Y,'.','SODP'+str(i),dir_+'/SODP'+str(i)+'.png')
        Area = calc_area_of_sodp(X,Y,i,channel) 
        # print(Area)
        return Area

In [7]:
seg = 10
samp = 256
non_overlap = 3
overlap = 7
step = non_overlap*samp
dist=seg*samp

In [8]:


# # file = open("/kaggle/input/chbmit-patient-wise-f1/merge_train_seizure.csv")
# # ts=pickle.load(file)
# # file.close()


# data=pd.read_csv("/kaggle/input/prev-chbmit-raw-data-sc/merge_all_nonseizure.csv",header=None)

# print(data.shape)
# data_shape = data.shape[0]
# # print(data_shape)
# full_sec = (data_shape//samp-(overlap))//non_overlap
# print(full_sec)

(2851328, 22)
3710


## Seizure Feature Creation Start

In [9]:
final_feature=[]
seizure_data = pd.read_csv("/kaggle/input/prev-chbmit-raw-data-sc/merge_all_seizure.csv",header=None)
seizure = seizure_data.shape[0]
print(seizure)
cnt =0
step = non_overlap*samp
for k in range (0,data_shape-(overlap*samp)-step,step):
# for k in range(0,2):
#     print(k//samp)
    cnt=cnt+1
    # print(cnt)
    feature = []
    ok = 1
    for i in range(0, 22):               
        imf = cal_IMF(k,i,seizure_data)
        imf = np.array(imf)
        if(imf.shape[0]<6):
            print(k,i,"boom")
            i=22
            ok = 0
            continue
        row = []
        for j in range(0,6):
            m = fluctuation_index(imf[j])
            n = statistics.variance(imf[j]) # entropy korte hbe
            o = SODP(imf[j], j, i)
            # final[k//step][i][3*j+0]=float(m)
            # final[k//step][i][3*j+1]=float(n)
            # final[k//step][i][3*j+2]=float(o)
            row.append(m)
            row.append(n)
            row.append(o)
        feature.append(row)
    if ok:
        feature=np.array(feature)
        final_feature.append(feature)
        print("final feature len ", len(final_feature))
                
print(cnt)

2851328
3710


## Non Seizure Feature creation 

In [ ]:
final_feature=[]
seizure_data = pd.read_csv("/kaggle/input/prev-chbmit-raw-data-sc/merge_all_nonseizure.csv",header=None)
seizure = seizure_data.shape[0]
print(seizure)
cnt =0
step = non_overlap*samp
for k in range (0,data_shape-(overlap*samp)-step,step):
# for k in range(0,2):
#     print(k//samp)
    cnt=cnt+1
    # print(cnt)
    feature = []
    ok = 1
    for i in range(0, 22):               
        imf = cal_IMF(k,i,seizure_data)
        imf = np.array(imf)
        if(imf.shape[0]<6):
            print(k,i,"boom")
            i=22
            ok = 0
            continue
        row = []
        for j in range(0,6):
            m = fluctuation_index(imf[j])
            n = statistics.variance(imf[j]) # entropy korte hbe
            o = SODP(imf[j], j, i)
            # final[k//step][i][3*j+0]=float(m)
            # final[k//step][i][3*j+1]=float(n)
            # final[k//step][i][3*j+2]=float(o)
            row.append(m)
            row.append(n)
            row.append(o)
        feature.append(row)
    if ok:
        feature=np.array(feature)
        final_feature.append(feature)
        print("final feature len ", len(final_feature))
                
print(cnt)

In [ ]:
final=np.array(final_feature)
final.shape

## Adding Lable

In [ ]:
temp = []
seizure = int(final.shape[0]/2)
for i in range(0, final.shape[0]):
    tt = final[i].reshape((396,))
    tt=tt.tolist()
    if(i<seizure):
        tt.append(1)
    else:
        tt.append(0)
    temp.append(tt)
temp=np.array(temp)
temp.shape

## Feature selection using MI

In [ ]:
cv = StratifiedKFold(n_splits=5, random_state=42, shuffle = True)

## data preprocessing
X = temp[:, :396]
y = temp[:,-1]


#####################################################################################
idx = 1
score = []

for train_index, test_index in cv.split(X, y):

    x_train,x_test = X[train_index],X[test_index]
    y_train,y_test = y[train_index],y[test_index]
#     x_train, x_val, y_train, y_val = train_test_split(x_train,y_train,test_size=0.1, stratify=y_train,random_state=42)
    
    file = open(f"/kaggle/input/5f-train-test-val-dataset/x_train{idx}.txt",'wb')
    pickle.dump(x_train, file)
    file.close()
    file = open(f"/kaggle/input/5f-train-test-val-dataset/x_test{idx}.txt","wb")
    pickle.dump(x_test, file)
    file.close()
    
    file = open(f"/kaggle/input/5f-train-test-val-dataset/y_train{idx}.txt","wb")
    pickle.dump(y_train, file)
    file.close()
    file = open(f"/kaggle/input/5f-train-test-val-dataset/y_test{idx}.txt","wb")
    pickle.dump(y_test, file)
    file.close()
    
    
    ################################# Mutual info based best feature selection #####################

    # Compute mutual information between each feature and the target variable for the training set
    mutual_info = mutual_info_classif(x_train, y_train)

    # Sort the features by mutual information in descending order
    indices = np.argsort(mutual_info)[::-1]
    file = open(f"/kaggle/input/5f-train-test-val-dataset/{idx}f_feature_matrix.txt","wb")
    pickle.dump(indices, file)
    file.close()
    
    idx = idx+1

## Model Train and Test

In [ ]:

####################################### init var #####################################################

idx=4
hn = 50
name = f"NN({hn})"
mi_mx = 100
acc_mx = -100
id = []
val = []

#################################train test val read for idx 1#########################################
###################         x            ###########################
file = open(f"/kaggle/input/5f-train-test-val-dataset/x_train{idx}.txt","rb")
x_train = pickle.load(file)
file.close()

file = open(f"/kaggle/input/5f-train-test-val-dataset/x_test{idx}.txt","rb")
x_test = pickle.load( file)
file.close()

###################         y            ###########################
file = open(f"/kaggle/input/5f-train-test-val-dataset/y_train{idx}.txt","rb")
y_train = pickle.load(file)
file.close()

file = open(f"/kaggle/input/5f-train-test-val-dataset/y_test{idx}.txt","rb")
y_test = pickle.load( file)
file.close()


file = open(f"/kaggle/input/5f-train-test-val-dataset/{idx}f_feature_matrix.txt","rb")
indices = pickle.load( file)
file.close()

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

######################################################################################################################################

for k in range (250, 397):
    
    id.append(k)

    top_features = indices[:k]
    X_train_top = x_train[:, top_features]
    X_test_top = x_test[:, top_features]
    print(X_train_top.shape)
    
    #################################  Standard Scaler #####################
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    # X = sc.fit_transform(X)
    X_train_top = sc.fit_transform(X_train_top)
    X_test_top = sc.fit_transform(X_test_top)

    ################################### reshape ##########################
    X_train_top=X_train_top.reshape(X_train_top.shape[0],X_train_top.shape[1],1)
    X_test_top=X_test_top.reshape(X_test_top.shape[0],X_test_top.shape[1],1)
   
    ################################# build model #####################
    from keras.models import Sequential
    from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, BatchNormalization

    model = Sequential()
    model.add(Dense(hn, input_dim=X_train_top.shape[1], activation='relu'))
    model.add(Dense(2, activation='softmax'))



    ################################# compile model #####################
    lr = .001
    batch_size = 128
    opt = tf.keras.optimizers.Adam(learning_rate = lr)
    model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])
    ################################# fit model #####################

    res_path = f'/kaggle/working/{name}_{idx}/mi_{k}/{idx}_MI_{k}.h5'
    mc = ModelCheckpoint(res_path, monitor='val_accuracy', mode='max', verbose=0, save_best_only=True)
#     es = EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=20)
    history = model.fit(X_train_top,y_train,validation_data = (X_test_top, y_test),batch_size=batch_size,epochs=400,verbose=0,callbacks=[mc])

    ################################# history data #####################
    train_loss  = history.history['loss']
    val_loss    = history.history['val_loss']
    accuracy    = history.history['accuracy']
    val_accuracy= history.history['val_accuracy']


    ################################# dump history data #####################
    x = np.array(list(range(1,len(train_loss)+1)))
    loss = 'model1'
    datas = {'epoch':x, 'Training loss':train_loss}
    lossepoch = pd.DataFrame(datas)
    lossepoch.to_csv(f'/kaggle/working/{name}_{idx}/mi_{k}/{loss}_train_loss.csv', index = False)

    datas = {'epoch':x, 'Test loss':val_loss}
    lossepoch = pd.DataFrame(datas)
    lossepoch.to_csv(f'/kaggle/working/{name}_{idx}/mi_{k}/{loss}_test_loss.csv', index = False)

    datas = {'epoch':x, 'Training accuracy':accuracy}
    lossepoch = pd.DataFrame(datas)
    lossepoch.to_csv(f'/kaggle/working/{name}_{idx}/mi_{k}/{loss}_train_accuracy.csv', index = False)

    datas = {'epoch':x, 'Test accuracy':val_accuracy}
    lossepoch = pd.DataFrame(datas)
    lossepoch.to_csv(f'/kaggle/working/{name}_{idx}/mi_{k}/{loss}_test_accuracy.csv', index = False)


    ########################## performance evalution #############################
    model.load_weights(res_path)
    scores = model.evaluate(X_test_top, y_test, verbose=1, batch_size=batch_size)
    pred=model.predict(X_test_top, verbose=0, batch_size=batch_size)
    pred=np.round(pred)
    #         error = 0
    #         for i in range (len(y_test)):
    #             # print(test_Y[i][0], " ", test_Y[i][1], "--> ", pred[i][0]," ", pred[i][1])
    #             if y_test[i][0]!=pred[i][0] or test_Y[i][1]!=pred[i][1]:
    #                 error = error+1
    #         print("error -> ",error)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    val.append(scores[1])
    os.rename(f'/kaggle/working/{name}_{idx}/mi_{k}', f'/kaggle/working/{name}_{idx}/mi_{k}_{round(scores[1],4)}')
    if (round(scores[1],4)>acc_mx):
        mi_mx = k
        acc_mx = round(scores[1],4)
    
os.rename(f'/kaggle/working/{name}_{idx}', f'/kaggle/working/{name}_{idx}_{mi_mx}_{acc_mx}')

## Result Store in excel Sheet

In [ ]:
df = pd.DataFrame(val,index=id, columns=[f'F{idx}'])
print(df)
df.to_excel(f"{name} f - {idx}.xlsx")